# YouTube Data API v3 Examples

In [1]:
from youtube.search import *
from youtube.videos import *
from youtube.comment_threads import *
from yt_api_key import key

## Search API

In [2]:
query = 'FTX'
search_params = {
    'key': key,
    'part': 'snippet',
    'type': 'video',
    'maxResults': 100,
    'relevanceLanguage': 'en',
    'publishedAfter': '2022-12-01' + 'T00:00:00Z',
    'publishedBefore': '2022-12-01' + 'T23:59:59Z',
    'order': 'viewCount'
}

In [3]:
search_results = await fetch_search_results(query, search_params)

l_video_ids = list(set([i['id']['videoId'] for i in search_results]))
print(f"Fetched {len(l_video_ids)} Video IDs")

Fetched 14 Video IDs


## Videos API

In [4]:
video_params = {
    'key': key,
    'part': 'id,statistics,topicDetails'
}

### Single Video

In [5]:
video_results = await fetch_videos('0Hxf4Vf54PI', video_params)

In [6]:
video_results

{'kind': 'youtube#video',
 'etag': 'T-XBuqdLqRU5wU1Ws5k3y05DE5U',
 'id': '0Hxf4Vf54PI',
 'statistics': {'viewCount': 616758,
  'likeCount': 3874,
  'favoriteCount': 0,
  'commentCount': 4456},
 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Society',
   'https://en.wikipedia.org/wiki/Television_program']}}

### Multiple Videos

In [7]:
# Concurrent
video_concurrent_results = await fetch_videos(l_video_ids, video_params)

print(f"Fetched data for {len(l_video_ids)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in video_concurrent_results.items():
    print(f"{k} | {v['statistics']['commentCount']}")

Fetched data for 14 videos
Video ID    | Comments
------------|------------
PamZCTjqR0k | 27
tC_orukfkGE | 27
5siCgOJXPdI | 63
0Hxf4Vf54PI | 4456
f26kRRKrlj8 | 17
HAnFA2FAl3o | 0
7QEbjRKbrjY | 2112
RDVXI84KSY8 | 126
ipcpgND2-AY | 2
IyoGdwVIwWw | 0
Bn0HOI9Qx0o | 52
8vwv23XYcrU | 13
Zc57tP7ODNQ | 179
U5poFl0S6JA | 11


In [8]:
# Sequential
comments_sequential_results = await fetch_videos(l_video_ids, video_params, sequential=True)

print(f"Fetched data for {len(comments_sequential_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in comments_sequential_results.items():
    print(f"{k} | {v['statistics']['commentCount']}")

Fetched data for 14 videos
Video ID    | Comments
------------|------------
PamZCTjqR0k | 27
tC_orukfkGE | 27
5siCgOJXPdI | 63
0Hxf4Vf54PI | 4456
f26kRRKrlj8 | 17
HAnFA2FAl3o | 0
7QEbjRKbrjY | 2112
RDVXI84KSY8 | 126
ipcpgND2-AY | 2
IyoGdwVIwWw | 0
Bn0HOI9Qx0o | 52
8vwv23XYcrU | 13
Zc57tP7ODNQ | 179
U5poFl0S6JA | 11


## CommentThreads API

In [9]:
comments_params = {
    'key': key,
    'part': 'id,replies,snippet',
    'order': 'time'
}

# Remove videos with less than n comments
n_comments_minimum = 0
l_video_ids = [v['id'] for k,v in video_concurrent_results.items() if v['statistics']['commentCount'] > n_comments_minimum]
print(f"{len(l_video_ids)} videos with more than {n_comments_minimum} comments")

12 videos with more than 0 comments


### Single Video Comments

In [10]:
comments_results = await fetch_comment_threads('0Hxf4Vf54PI', comments_params)

print(f"Fetched {len(comments_results)} comments for Video ID `0Hxf4Vf54PI`")

Fetched 3390 comments for Video ID `0Hxf4Vf54PI`


### Multiple Videos Comments

In [11]:
# Concurrent
comments_concurrent_results = await fetch_comment_threads(l_video_ids, comments_params)

print(f"Fetched comments for {len(comments_concurrent_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for i in comments_concurrent_results.keys():
    print(f"{i} | {len(comments_concurrent_results[i])}")

Fetched comments for 12 videos
Video ID    | Comments
------------|------------
PamZCTjqR0k | 23
tC_orukfkGE | 25
5siCgOJXPdI | 52
0Hxf4Vf54PI | 3390
f26kRRKrlj8 | 15
7QEbjRKbrjY | 1799
RDVXI84KSY8 | 85
ipcpgND2-AY | 1
Bn0HOI9Qx0o | 43
8vwv23XYcrU | 11
Zc57tP7ODNQ | 119
U5poFl0S6JA | 3


In [12]:
# Sequential
comments_sequential_results = await fetch_comment_threads(l_video_ids, comments_params, sequential=True)

print(f"Fetched comments for {len(comments_sequential_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for i in comments_sequential_results.keys():
    print(f"{i} | {len(comments_sequential_results[i])}")

Fetched comments for 12 videos
Video ID    | Comments
------------|------------
PamZCTjqR0k | 23
tC_orukfkGE | 25
5siCgOJXPdI | 52
0Hxf4Vf54PI | 3390
f26kRRKrlj8 | 15
7QEbjRKbrjY | 1799
RDVXI84KSY8 | 85
ipcpgND2-AY | 1
Bn0HOI9Qx0o | 43
8vwv23XYcrU | 11
Zc57tP7ODNQ | 119
U5poFl0S6JA | 3
